# Script for training with decoupled shape parameters

In [ ]:
from decalib.deca import DECA
from decalib.datasets import datasets 
from decalib.utils import util
from decalib.utils.config import cfg as deca_cfg
from skimage.io import imread
from tqdm import tqdm
import torch
import numpy as np
import mediapipe as mp
import os
import cv2
import trimesh
import matplotlib.pyplot as plt
import logging
import ipywidgets as widgets
from IPython.display import display

Function to get the Shape parameters for an image -> we need this later

In [ ]:
def get_shape_params(image_path: str, device='cuda'):
    """Function for getting the shape paramters of an image
    """
    # Initialization
    deca = DECA()

    #Get the data in the right format
    test_img = datasets.TestData(image_path)
    deca_cfg.model.use_tex = False
    deca_cfg.model.extract_tex = False
    #Get the codedictionarys
    for i in tqdm(range(len(test_img))):
        name = test_img[i]['imagename']
        images_l = test_img[i]['image'].to(device)[None,...]
        with torch.no_grad(): 
            codedict = deca.encode(images_l)
            
    return codedict['shape']

For every image, we need to get the shape parameters 

Get the landmarks

In [ ]:
images_folder = '/home/dietrich/Testing/DECA/Dataset/VGGFACE/Images/VGGface2_None_norm_512_true_bygfpgan'
lmks_folder = '/home/dietrich/Testing/DECA/Dataset/VGGFACE/lmks_train'
shape_folder = '/home/dietrich/Testing/DECA/Dataset/VGGFACE/shape_train'
img_name_list = []

total_steps = 9279
counter = 0

progress_slider = widgets.IntSlider(min=0, max=total_steps, value=counter, description='Progress')
logging.basicConfig(filename='/home/dietrich/Testing/DECA/Dataset/VGGFACE/lmks_and_shape_processing_6_imgs.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def update_slider(value):
    progress_slider.value = value
display(progress_slider)

lms_478_to_68 = [
                162,234,93,58,172,136,149,148,152,377,378,365,397,288,323,454,389,71,63,105,66,107,336,
                296,334,293,301,168,197,5,4,75,97,2,326,305,33,160,158,133,153,144,362,385,387,263,373,
                380,61,39,37,0,267,269,291,405,314,17,84,181,78,82,13,312,308,317,14,87
            ]

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
                static_image_mode = True,
                max_num_faces = 1,
                refine_landmarks=False,
                )

expected_landmarks_count = 34 # Half should be detected
# Through all folders in the Dataset Path
for individ__name in os.listdir(images_folder):
    individ__path = os.path.join(images_folder, individ__name)

    if os.path.isdir(individ__path):
        lmks_individ_path = os.path.join(lmks_folder, individ__name)
        if not os.path.exists(lmks_individ_path):
            os.makedirs(lmks_individ_path)
        # e.g. K - number of images per person
        img_num = 6
        # counting valid images and break, if enough images are taken
        img_list_individual = []
        # For one individual (one folder) going through the images, until 6 are detected
        for image_name in os.listdir(individ__path):
            image_path = os.path.join(individ__path, image_name)

            if image_name.endswith('.jpg'):
                fn, ext = os.path.splitext(os.path.basename(image_name))

                #Process Mediapipe
                image = cv2.imread(image_path)
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype('uint8')

                results = face_mesh.process(image_rgb)
                
                # Excactly one person should be detected, else the image is not used
                if results.multi_face_landmarks and len(results.multi_face_landmarks) == 1:
                    landmarks = results.multi_face_landmarks[0].landmark
                    selected_lm = [landmarks[i] for i in lms_478_to_68] 
                    selected_lm = [[lm.x * image.shape[1], lm.y * image.shape[0]] for lm in selected_lm]

                    # Half of the face need to be detected, else another image is used
                    if len(selected_lm) >= expected_landmarks_count:
                        np.save(os.path.join(lmks_individ_path, fn + "_68kpts.npy") ,selected_lm)
                        img_list_individual.append(os.path.join(individ__name, fn))
                    else:
                        logging.info(f'Not used: Detected less than 35 Landmarks: {individ__path}/{image_name}')
                else:
                    logging.info(f'Not used: Detected none or more than one face: {individ__path}/{image_name}')

            #If we got 6 Images, the next individual will be landmarked
            if len(img_list_individual) == img_num:
                img_name_list.append(img_list_individual)
                np.save("/home/dietrich/Testing/DECA/Dataset/VGGFACE/"+"data_names_6_per_individual", img_name_list)
                break
    counter +=1
    update_slider(counter)
    
print("Landmark Detection and Image Selection Done")

Test, if getting shape parameters is working the right way

In [ ]:
datafile = '/home/dietrich/Testing/DECA/Dataset/VGGFACE/data_names_6_per_individual.npy'
data_lines = np.load(datafile).astype('str')

In [ ]:
shape_list = []
base_path = '/home/dietrich/Testing/DECA/Dataset/VGGFACE/Images/VGGface2_None_norm_512_true_bygfpgan'

idx = 0
for i in range(6):
    name = data_lines[idx, i]
    image_path = os.path.join(base_path, name + '.jpg') 
    shape_params = get_shape_params(image_path)
    shape_list.append(shape_params)

shape_2d = torch.stack(shape_list)
mean_shape = torch.mean(shape_2d, dim=0).cpu().detach().numpy()
mean_shape_list = [mean_shape for _ in shape_list]


shapearray = torch.from_numpy(np.array(mean_shape_list)).type(dtype = torch.float32) #K,100

print(shapearray)

Actual Training

In [ ]:
from decalib.deca import DECA
from decalib.trainer import Trainer
from decalib.utils.config import cfg
import yaml
import shutil
import torch.backends.cudnn as cudnn
import os
import torch
import torch.multiprocessing as mp

In [ ]:
from decalib.models.encoders import ResnetEncoder
import numpy as np
from decalib.datasets import datasets 
import torch
import os
from decalib.utils import util

n_shape = 100
n_tex = 50
n_exp = 50
n_pose = 6
n_cam = 3
n_light = 27

model_path = "/home/dietrich/Testing/DECA/DECA/data/deca_model.tar"
num_params = n_shape+n_tex+n_exp+n_pose+n_cam+n_light
E_flame  = ResnetEncoder(outsize=num_params).to('cuda')
param_dict = {'shape': n_shape, 'tex': n_tex, 'exp': n_exp, 'pose': n_pose, 'cam': n_cam, 'light': n_light}
checkpoint = torch.load(model_path)
util.copy_state_dict(E_flame.state_dict(), checkpoint['E_flame'])
E_flame.eval()

def encode(imgs_batch):
        with torch.no_grad():
                parameters = E_flame(imgs_batch)
        code_dict__ = decompose_code(parameters, param_dict)
        code_dict__['images'] = imgs_batch
        return code_dict__

def decompose_code(concatCode, num_dict__):
        ''' Convert a flattened parameter vector to a dictionary of parameters
        code_dict.keys() = [('shape',) 'tex', 'exp', 'pose', 'cam', 'light']
        '''
        code_dict_ = {}
        start_ind_ = 0
        for key in num_dict__:
            end_ind___      = start_ind_+int(num_dict__[key])
            code_dict_[key] = concatCode[:, start_ind_:end_ind___]
            start_ind_      = end_ind___
            if key == 'light':
                code_dict_[key] = code_dict_[key].reshape(code_dict_[key].shape[0], 9, 3)
        return code_dict_

In [ ]:
data_list = np.load('/home/dietrich/Testing/DECA/Dataset/VGGFACE/data_names_6_per_individual.npy').astype('str')
image_folder = '/home/dietrich/Testing/DECA/Dataset/VGGFACE/Images/VGGface2_None_norm_512_true_bygfpgan'
shape_folder = '/home/dietrich/Testing/DECA/Dataset/VGGFACE/shape_train'
shape_mean_folder = '/home/dietrich/Testing/DECA/Dataset/VGGFACE/shape_train_mean'

imagepath = [image_folder + '/' + data_name + '.jpg' for data_name in data_list[0, :]]

Get Shape Parameters

In [ ]:
num=0
for idx in range(data_list.shape[0]):
    folder_path = os.path.join(shape_folder, data_list[idx, 0].split('/')[0])
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    folder_path_mean = os.path.join(shape_mean_folder, data_list[idx, 0].split('/')[0])
    if not os.path.exists(folder_path_mean):
        os.makedirs(folder_path_mean)
    imagepath = [image_folder + '/' + data_name + '.jpg' for data_name in data_list[idx, :]]
    testdata__ = datasets.TestData(imagepath)
    shape_array = []
    for i in range(len(imagepath)):
        imgs_batch = testdata__[i]['image'].to('cuda')[None,...]
        codedict = encode(imgs_batch)
        np.save(shape_folder + '/' + data_list[idx, i] + '.npy', codedict['shape'].cpu().numpy())
        shape_array.append(codedict['shape'].cpu().numpy())
    mean_shape = np.mean(np.array(shape_array), axis=0)
        
    np.save(shape_mean_folder + '/' + data_list[idx, 0].split('/')[0] + '/shape_mean.npy', mean_shape)
    num+=1
    print(num)


In [ ]:
import numpy as np

data_list = np.load('/home/dietrich/Testing/DECA/Dataset/VGGFACE/data_names_6_per_individual.npy').astype('str')

data_list_valid = data_list[:20, :]

data_list_train = data_list[20:, :]

np.save('/home/dietrich/Testing/DECA/Dataset/VGGFACE/data_names_6_per_individual_train.npy', data_list_train)

np.save('/home/dietrich/Testing/DECA/Dataset/VGGFACE/data_names_6_per_individual_valid.npy', data_list_valid)

In [ ]:
exec $SHELL

sbatch train.sh

squeue #Aktueller stand